In [ ]:
from collections import Counter
import math
from typing import List
from os.path import join
import pickle
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
nice_fonts = {
        # Use LaTeX to write all text
        "text.usetex": True,
        "font.family": "serif",
        # Use 10pt font in plots, to match 10pt font in document
        "axes.labelsize": 10,
        "font.size": 10,
        # Make the legend/label fonts a little smaller
        "legend.fontsize": 8,
        "xtick.labelsize": 8,
        "ytick.labelsize": 8,
}

mpl.rcParams.update(nice_fonts)

def set_size(width: float = 347.0, fraction: float = 1., subplot: List[int] = [1, 1]):
    """ Set aesthetic figure dimensions to avoid scaling in latex.

    Parameters
    ----------
    width: float
            Width in pts
            Default value 347.12354 is textwidth for Springer llncs
    fraction: float
            Fraction of the width which you wish the figure to occupy

    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches

    From: https://jwalton.info/Embed-Publication-Matplotlib-Latex/
    """
    # Width of figure
    fig_width_pt = width * fraction

    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    golden_ratio = (5**.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio * (subplot[0] / subplot[1])

    fig_dim = (fig_width_in, fig_height_in)

    return fig_dim

In [ ]:
def load_data(dataset):
    return pickle.load( open( '%s_ultimate_perf_dict.pickle' % dataset, 'rb' ) ), pd.read_csv( '%s_bin_infos.csv' % dataset )

path = ''
prefix = ''
results = {}
for dataset in ['abalone', 'concreteStrength', 'delta_ailerons', 'boston',
                'available_power', 'servo', 'bank8FM', 'machineCpu', 'airfoild',
                'a2', 'a3', 'a1', 'cpu_small', 'acceleration', 'maximal_torque',
                'a4', 'a5', 'a7', 'fuel_consumption_country', 'a6',
                'normal', 'dnormal', 'pareto', 'rpareto']:
    results[dataset] = load_data(prefix + dataset, path=path)

In [ ]:
def sorted_mixed(l):
    other_types = [rk for rk in l if type(rk[1]) != float]
    sorted_floats = sorted([rk for rk in l if type(rk[1]) == float])
    return sorted_floats + other_types    

In [ ]:
def model_sig_str(model):
    return str(model) + ' sig.'

In [ ]:
def create_bin_rank_mapping(dataset_bin_info):
    dataset_bin_info = dataset_bin_info.sort_values(by=['count'])
    
    # Create mappings from bin to bin rank and vice versa
    bin_to_rank = {}
    rank_to_bin = {}
    for i, bin_row in enumerate(dataset_bin_info.iterrows()):
        bin_to_rank[(bin_row[1]['bin_low'], bin_row[1]['bin_high'])] = i
        rank_to_bin[i] = (bin_row[1]['bin_low'], bin_row[1]['bin_high'])
        
    return bin_to_rank, rank_to_bin   

In [ ]:
metric = 'rmse'
# metric = 'mean_absolute_error'
fraction = 1.4
models = [a/10 for a in list(range(0,21,1))]
datasets = ['normal', 'dnormal', 'pareto', 'rpareto']

fig, axs = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=False, figsize=set_size(fraction=fraction, subplot=[2, 2]))
linestyles = ['-', '--', ':']

for i, dataset in enumerate(datasets):
    res, bin_infos = results[dataset]
    bin_infos = bin_infos.sort_values(by=['count'])
    
    # Create mappings from bin to bin rank and vice versa
    bin_to_rank, rank_to_bin = create_bin_rank_mapping(bin_infos)

    for bin_rank in range(len(bin_infos)):
        b = rank_to_bin[bin_rank]
        xs = sorted(models)
        ys = [res[metric][b][(m, m)][0] for m in xs]
        row = i//2
        col = i%2
        axs[row][col].plot(xs, ys, label=f'Bin Rank {int(bin_rank)+1}', color=str(1/(bin_rank+1.5)), linestyle=linestyles[bin_rank % len(linestyles)])
        axs[row][col].set_title(dataset)
        axs[row][col].set_xticks(xs[::5])

        if i in [0, 2]:
            axs[row][col].set_ylabel('RMSE')
        if i in [2, 3]:
            axs[row][col].set_xlabel('$\\alpha$')

axs[0][0].legend(ncol=1, fontsize='xx-small')
plt.subplots_adjust(hspace=0.25)

fig.savefig(join('..', 'plots', 'synth_datasets_results_separate.pdf'), format='pdf', bbox_inches='tight')

In [ ]:
synth = False

metric = 'rmse'
# metric = 'mean_absolute_error'
fraction = 0.55

if synth:
    models = [a/10 for a in list(range(0,21,1))]
    datasets = ['normal', 'dnormal', 'pareto', 'rpareto']
    # Check significance per model only against the baseline of doing nothing
    sig_against_all = False
    # Show only significant wins
    show_only_sig_wins = False
    ignore_sig = True
    remove_empty_from_legend = True
    legend_cols = 5
    edge_colors = [None, 'gray', 'gray']
else:
    models = [0.0, 1.0, '{SMOGN}']
    # models = [0.0, 1.0, '{SMOGN_DW}']
    datasets = ['abalone', 'concreteStrength', 'delta_ailerons', 'boston',
                'available_power', 'servo', 'bank8FM', 'machineCpu', 'airfoild',
                'a2', 'a3', 'a1', 'cpu_small', 'acceleration', 'maximal_torque',
                'a4', 'a5', 'a7', 'fuel_consumption_country', 'a6']
    # Check significance per model against all other models
    sig_against_all = True
    # Show both significant and insignificant wins
    show_only_sig_wins = False
    ignore_sig = False
    remove_empty_from_legend = False
    legend_cols = 3
    edge_colors = [None]
        
wins_per_bin_rank = {0: [], 1: [], 2: [], 3: [], 4: []}
# wins_per_bin_rank looks like this: {0: [0.0, 0.5, 0.0, 1.0, 'SMOGN', 0.0, ...]}
for dataset in datasets:
    res, bin_infos = results[dataset]
    bin_infos = bin_infos.sort_values(by=['count'])
    
    # Create mappings from bin to bin rank and vice versa
    bin_to_rank, rank_to_bin = create_bin_rank_mapping(bin_infos)
        
    for bin_rank in range(len(bin_infos)):
        b = rank_to_bin[bin_rank]
        model_to_score = {}
        for m in models:
            model_to_score[m] = res[metric][b][(m, m)][0]
            
        if all([math.isnan(score) for score in model_to_score.values()]):
            best_model = 'No Datapoints'
            significant = [False]
        else:
            
            while True:
                best_model = min(model_to_score, key=model_to_score.get)
                if ignore_sig:
                    significant = [False]
                    break
                if sig_against_all:
                    # Check if significant against all other models
                    significant = [res[metric][b][(best_model, other_model)][6] for other_model in models if best_model != other_model]
                else:
                    # Check if significant against no method
                    significant = [res[metric][b][(best_model, 0.0)][6]]
                if show_only_sig_wins:
                    if significant:
                        break
                        
                    model_to_score[best_model] = None
                else:
                    break
    
        if bin_rank == 0:
            print('bin rank 0 at', dataset, 'won by', best_model)

        best_model_str = model_sig_str(best_model) if all(significant) else str(best_model)
        wins_per_bin_rank[bin_rank].append(best_model_str)
        
counted_wins = {br: Counter(wins_per_bin_rank[br]) for br in wins_per_bin_rank.keys()}
print(counted_wins)

def hor_bar_plot(results, category_names, category_colors=None, frac=1):
    """
    Parameters
    ----------
    results : dict
        A mapping from question labels to a list of answers per category.
        It is assumed all lists contain the same number of entries and that
        it matches the length of *category_names*.
    category_names : list of str
        The category labels.
    """
    labels = list(results.keys())
    data = np.array(list(results.values()))
    data_cum = data.cumsum(axis=1)
    if category_colors is None:
        cmap = 'viridis'
        category_colors = plt.get_cmap(cmap)(
            np.linspace(0.0, 1.0, data.shape[1]))

    fig, ax = plt.subplots(figsize=set_size(fraction=frac))
    ax.invert_yaxis()
    ax.set_xlim(0, np.sum(data, axis=1).max())
    
    ax.set_xticks([]) 
    
    ax.set_ylabel('Bin Ranks')
    
    hatch_styles = [None]
    line_styles = ['-', '--', '-']

    for i, (colname, color) in enumerate(zip(category_names, category_colors)):
        widths = data[:, i]
        starts = data_cum[:, i] - widths
        ax.barh(labels, widths, left=starts, height=0.5, edgecolor=edge_colors[i%len(edge_colors)], linestyle=line_styles[i%len(line_styles)],# fill=False,
                label=colname, color=color, hatch=hatch_styles[i%len(hatch_styles)])
        xcenters = starts + widths / 2

        r, g, b, _ = color
        text_color = 'white' if r * g * b < 0.5 else 'darkgrey'
        for y, (x, c) in enumerate(zip(xcenters, widths)):
            if c != 0:
                ax.text(x, y, str(int(c)), ha='center', va='center',
                        color=text_color)
    ax.legend(ncol=legend_cols ,bbox_to_anchor=(0.5, 1), loc='lower center', fontsize='xx-small')

    return fig, ax

nice_names = {
    '0.0': 'None',
#     '1.0': '$\\alpha = 1.0$',
    '1.0': 'DenseLoss',
    '{SMOGN}': 'SMOGN',
    '{SMOGN_DW}': 'SMOGN-DW',
}

category_names = []
nice_category_names = []
for m in models:
    if not show_only_sig_wins:
        category_names.append(str(m))
    if not ignore_sig:
        category_names.append(model_sig_str(m))
    if synth:
        if not show_only_sig_wins:
            nice_category_names.append(str(m))
        if not ignore_sig:
            nice_category_names.append(model_sig_str(str(m)))
    else:
        if not show_only_sig_wins:
            nice_category_names.append(nice_names[str(m)])
        if not ignore_sig:
            nice_category_names.append(model_sig_str(nice_names[str(m)]))

if synth:
    category_colors = None
else:
    sig_a = 1.0
    insig_a = 0.5
    category_colors = [(0.12,0.47,0.7,insig_a), (0.12,0.47,0.7,sig_a), (0.0,0.0,0.0,insig_a), (0.0,0.0,0.0,sig_a), (1.0,0.5,0.05,insig_a), (1.0,0.5,0.05,sig_a), (0.5,0.5,0.05,insig_a), (0.5,0.5,0.05,sig_a)]

bin_names = {
    0: '1',
    1: '2',
    2: '3',
    3: '4',
    4: '5',
}

res_to_plot = {}
for br, win_counts in counted_wins.items():
    res_to_plot[bin_names[br]] = []
    for cn in category_names:
        wc = win_counts[cn] if cn in win_counts.keys() else 0
        res_to_plot[bin_names[br]].append(wc)
        
# Remove models that won't show up in plot
if remove_empty_from_legend:
    win_indices_to_remove = set()
    for i, m in enumerate(category_names):
        bin_wins = 0
        for br, wins in res_to_plot.items():
            bin_wins += wins[i]

        if bin_wins == 0:
            category_names[i] = None
            nice_category_names[i] = None
            for br, wins in res_to_plot.items():
                win_indices_to_remove.add(i)

    category_names = [cn for cn in category_names if cn is not None]
    nice_category_names = [cn for cn in nice_category_names if cn is not None]
    for br, wins in res_to_plot.items():
        for i in sorted(win_indices_to_remove, reverse=True):
            del res_to_plot[br][i]

fig, ax = hor_bar_plot(res_to_plot, nice_category_names, category_colors, frac=fraction)
if synth:
    ax.text(-0.8, 0, 'Least', fontsize='x-small')
    ax.text(-0.9, 0.25, 'common', fontsize='x-small')
    ax.text(-0.8, 3.95, 'Most', fontsize='x-small')
    ax.text(-0.9, 4.25, 'common', fontsize='x-small')
else:
    ax.text(-4.9, 0, 'Least', fontsize='x-small')
    ax.text(-5.7, 0.25, 'common', fontsize='x-small')
    ax.text(-4.9, 3.95, 'Most', fontsize='x-small')
    ax.text(-5.7, 4.25, 'common', fontsize='x-small')
plot_name = 'synth_datasets_results.pdf' if synth else prefix + 'real_datasets_results.pdf'
fig.savefig(join('..', 'plots', plot_name), format='pdf', bbox_inches='tight')